In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [84]:
leaves_df = pd.read_csv('leaves.csv')
holidays_df = pd.read_csv('holidays.csv')

In [85]:
leaves_df.head()

,Employee_No,leave_date,Type,Applied Date,Remarks,apply_type
0,347,1/29/2020,Half Day,1/4/2022 15:33,\N,Anual
1,347,1/29/2020,Half Day,1/4/2022 15:33,\N,Anual
2,347,1/29/2020,Half Day,1/4/2022 15:36,\N,Anual
3,347,1/29/2020,Half Day,1/4/2022 15:36,\N,Anual
4,347,1/29/2020,Half Day,1/4/2022 18:02,\N,Anual


In [86]:
holidays_df.head()

,2017-05-07
0,2017-05-14
1,2017-05-21
2,2017-05-28
3,2017-06-04
4,2017-06-11


In [87]:
#concat two pandas series in axis 0
holidays = pd.concat([pd.Series(holidays_df.columns[0]), pd.Series(holidays_df.iloc[:,0])], axis=0)
#holidays as dataframe
holidays = pd.DataFrame(holidays)
#rename column
holidays.columns = ['Date']
#convert to datetime
holidays['Date'] = pd.to_datetime(holidays['Date'])
#reset index
holidays_df = holidays.reset_index(drop=True)
holidays_df.head()


,Date
0,2017-05-07
1,2017-05-14
2,2017-05-21
3,2017-05-28
4,2017-06-04


In [88]:
leaves_df['leave_date'] = pd.to_datetime(leaves_df['leave_date'])
leaves_df['Applied Date'] = pd.to_datetime(leaves_df['Applied Date'])
leaves_df['Applied Date'] = leaves_df['Applied Date'].dt.date
leaves_df['Applied Date']

0       2022-01-04
1       2022-01-04
2       2022-01-04
3       2022-01-04
4       2022-01-04
           ...    
1013    2023-01-31
1014    2023-01-31
1015    2023-02-03
1016    2023-02-03
1017    2023-02-03
Name: Applied Date, Length: 1018, dtype: object

In [89]:
#remove the records in leaves_df leave_date column with dates in holidays_df
leaves_df = leaves_df[~leaves_df['leave_date'].isin(holidays_df['Date'])]
leaves_df.head()

,Employee_No,leave_date,Type,Applied Date,Remarks,apply_type
0,347,2020-01-29,Half Day,2022-01-04,\N,Anual
1,347,2020-01-29,Half Day,2022-01-04,\N,Anual
2,347,2020-01-29,Half Day,2022-01-04,\N,Anual
3,347,2020-01-29,Half Day,2022-01-04,\N,Anual
4,347,2020-01-29,Half Day,2022-01-04,\N,Anual


In [90]:
# remove duplicates leaves_df
leaves_df = leaves_df.drop_duplicates()
#remove duplicates leaves_df Employee_No, leave_date
leaves_df = leaves_df.drop_duplicates(subset=['Employee_No', 'leave_date'])

In [91]:
#drop columns Applied Date, Remarks
leaves_df = leaves_df.drop(['Applied Date', 'Remarks','leave_date'], axis=1)
leaves_df.head()

,Employee_No,Type,apply_type
0,347,Half Day,Anual
5,1961,Full Day,Anual
6,1961,Full Day,Anual
7,2346,Full Day,Anual
8,2346,Full Day,Anual


In [92]:

leaves_df.shape

(947, 3)

In [93]:
# one hot encoding for Type and apply type
leaves_df = pd.get_dummies(leaves_df, columns=['Type', 'apply_type'])

leaves_df = leaves_df.reset_index(drop=True)
leaves_df = leaves_df.iloc[:,:].astype(int)
leaves_df.head()

,Employee_No,Type_Full Day,Type_Half Day,apply_type_Anual,apply_type_Casual
0,347,0,1,1,0
1,1961,1,0,1,0
2,1961,1,0,1,0
3,2346,1,0,1,0
4,2346,1,0,1,0


In [94]:
# group by Employee_No and sum
leaves_df = leaves_df.groupby('Employee_No').sum()
leaves_df.head()


,Type_Full Day,Type_Half Day,apply_type_Anual,apply_type_Casual
Employee_No,,,,
347,16,6,11,11
348,5,7,5,7
351,4,6,0,10
356,6,6,0,12
373,15,6,14,7


In [95]:
leaves_df.shape

(69, 4)

In [96]:
leaves_df.to_csv('leaves_preprocessed.csv', index=True)